In [16]:
import requests
import json
import os
from urllib.request import urlretrieve
import zipfile
import glob
import pandas as pd
import re
from memory_profiler import memory_usage
import pyarrow.dataset as ds
import pyarrow.feather as feather
import pyarrow.parquet as pq
import dask.dataframe as dd

In [2]:
%load_ext memory_profiler

### Downloading the data

In [3]:
article_id = 14096681
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figsharerainfall/"
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)
files = data["files"]
files_to_dl = ["data.zip"]
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

### Combining data CSVs

In [4]:
%%time
%memit

files = glob.glob('figsharerainfall/*.csv')

df = pd.concat((pd.read_csv(file, header=0, index_col=0)
                .assign(model=re.search(r'/(.*)_d', file)[1])
                for file in files))

df.to_csv("figsharerainfall/combined_data.csv")

peak memory: 95.25 MiB, increment: 0.25 MiB
CPU times: user 6min 13s, sys: 15.3 s, total: 6min 29s
Wall time: 6min 40s


In [5]:
%%sh
du -sh figsharerainfall/combined_data.csv

5.6G	figsharerainfall/combined_data.csv


### Load the combined CSV to memory

In [6]:
%%time
df = pd.read_csv("figsharerainfall/combined_data.csv")

CPU times: user 58.3 s, sys: 11.8 s, total: 1min 10s
Wall time: 1min 13s


### Changing `dtype` of data

In [7]:
%%time
%%memit

dataset = ds.dataset("figsharerainfall/combined_data.csv", format="csv")
table = dataset.to_table()

peak memory: 5249.03 MiB, increment: 1512.00 MiB
CPU times: user 19.9 s, sys: 11.3 s, total: 31.2 s
Wall time: 27.9 s


#### Using feather:

In [8]:
%%time
feather.write_feather(table, 'figsharerainfall/combined_data.feather')

CPU times: user 4.55 s, sys: 12.7 s, total: 17.2 s
Wall time: 7.15 s


In [9]:
%%sh
du -sh figsharerainfall/combined_data.feather

1.0G	figsharerainfall/combined_data.feather


#### Using parquet:

In [10]:
%%time 
pq.write_table(table, 'figsharerainfall/combined_data.parquet')

CPU times: user 9.87 s, sys: 1.03 s, total: 10.9 s
Wall time: 10.9 s


In [11]:
%%sh
du -sh figsharerainfall/combined_data.parquet

544M	figsharerainfall/combined_data.parquet


### EDA

In [12]:
df.shape

(62513863, 7)

In [13]:
df.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1,1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
2,1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
3,1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
4,1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM
